## Polarity is all you need to learn and transfer faster
This notebook is a tutorial for reproducing the computer vision results in **Polarity is all you need to learn and transfer faster**.  
* Data used in this notebook can be downloaded from OSF following this anonymized [link](https://osf.io/f9wtc/?view_only=61b71c37306a41209da0eb1c35dbf8d0)
* Pre-trained AlexNet weights were obtainer from [here](https://www.cs.toronto.edu/~guerzhoy/tf_alexnet/). Please download the tf2 version before running the experiments code
* All experiments and analysis were performed within the following docker environment, which could be setup as following. 
```setup
docker build -t weightpolarity .
docker run --gpus all -v ${PWD}:/workspace --name weightpolarity weightpolarity
```

## Computer vision experiments

## Experiments

* The experiments are set up to run in background in paralell. 
* You may choose to change the training sample size (sample_size_list) and epoch number (num_epoch) configurations in batchcis.py. The current setup is for paper reproduction. 
* You are suggested to run below lines in command line. 

In [ ]:
! python batchcis.py --num_run=20 --baseFName="fashion_mnist" --num_epoch=100 --ckpt_freq=100 --doRandInit=2
! python batchcis.py --num_run=20 --baseFName="cifar10" --num_epoch=100 --ckpt_freq=100 --doRandInit=2

### Consolidate all data
* Below two cells, will collect all experimental data into "epoch_acc_loss.pkl" file which are accessible from the above OSF [link](https://osf.io/f9wtc/?view_only=61b71c37306a41209da0eb1c35dbf8d0). The first one is for fashion_mnist, second is for cifar10

In [ ]:
! python doPlots.py --num_run=20 --baseFName='/Users/alice/cis/wp_CV/fashion_mnist' --resetType='posRand' --numEpoch=100

In [ ]:
! python doPlots.py --num_run=20 --baseFName='/Users/alice/cis/wp_CV/cifar10' --resetType='posRand' --numEpoch=100

## Analysis

### 1.Preparation - load necessary packages and helper functions

In [ ]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import matplotlib as matplotlib

In [ ]:
# white background
plt.rcParams.update({
    "lines.color": "black",
    "patch.edgecolor": "black",
    "text.color": "black",
    "axes.facecolor": "black",
    "axes.edgecolor": "black",
    "axes.labelcolor": "black",
    "xtick.color": "black",
    "ytick.color": "black",
    "grid.color": "lightgray"})

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from plots import dict_to_query, plot_median_plus_example, simpleaxis, plot_sem, plot_diff_plus_mannwhitneyu

In [ ]:
def paint_color(x, y):
    if x > 0:
        if y > 0:
            return np.array(3)
        else:
            return np.array(2)
    else:
        if y > 0:
            return np.array(1)
        else:
            return np.array(0)
np_paint_color = np.vectorize(paint_color, otypes = [np.ndarray])

### 2.Load Data
* Here, we will use experiment data presented in the manuscript. They are presented in the main text, each experiment was ran for 20 repeats

In [ ]:
metrics = np.load('epoch_acc_loss.pkl', allow_pickle=True)

In [ ]:
# Decided to still mannually define here so that it can be cross-checked with the loaded data
config = {}
config['batch_size'] = 1000

doWeightFreeze_list = [True, False]
sample_size_list = np.concatenate(([100,250,500,750], np.arange(1,7)*1000)) # fashion-MNIST
# sample_size_list = np.array([100,250,500,750,1000,2500,5000,10000,25000,50000]) # cifar10
num_epoch_list = [50] * len(sample_size_list)
networkType_list = ['pretrained', 'vanilla', 'finetune']

numVal = 1000
numRun = 20 
resetType_list = ['posRand']

### 3.**DATA: Statistical Efficiency** - Figure 2 & 3 first column

#### Load functions

In [ ]:
def get_converge_epoch(x):
    epoch_num = np.squeeze(np.where(x == np.amin(x)))
    if not len(epoch_num.shape)==0:
        epoch_num = epoch_num[0]
    return epoch_num

In [ ]:
def get_acc_at_conver(val_acc, val_loss):
    # both are matrices with rows as run
    epoch_at_conver = np.apply_along_axis(get_converge_epoch, 1, val_loss)
    acc_at_conver = np.squeeze(np.take_along_axis(val_acc, np.expand_dims(epoch_at_conver, axis=1), axis=1))
    # print(epoch_at_conver.shape)
    # print(acc_at_conver.shape)    
    return acc_at_conver, epoch_at_conver

In [ ]:
def get_conver(metrics, doWeightFreeze_list, resetType_list, networkType, numRun):
    conver = pd.DataFrame(index=range(len(doWeightFreeze_list)*len(resetType_list)), columns=["typeStr", "resetType", "networkType", "acc", "epoch"])

    for ridx, resetType in enumerate(resetType_list):
        for didx, doWeightFreeze in enumerate(doWeightFreeze_list):
            df_loc = ridx*len(doWeightFreeze_list)+didx
            if doWeightFreeze:
                typeStr = 'freeze'
                typeStr_dict = '\'freeze\''
            else:
                typeStr = 'liquid'
                typeStr_dict = '\'liquid\''
            conver['typeStr'][df_loc] = typeStr
            conver['resetType'][df_loc] = resetType
            conver['networkType'][df_loc] = networkType

            # get different sample sizes into list
            page_dict = {'networkType':'\''+ networkType +'\'', 'typeStr':typeStr_dict, 'resetType':'\''+ resetType +'\''}
            val_loss = metrics.query(dict_to_query(page_dict))['validation_loss'].tolist()
            val_acc = metrics.query(dict_to_query(page_dict))['validation_acc'].tolist()
            # print(val_acc[0].shape)

            acc_at_conver = np.empty(shape=(len(sample_size_list), numRun))
            epoch_at_conver = np.empty(shape=(len(sample_size_list), numRun))
            for idx, (this_val_loss, this_val_acc) in enumerate(zip(val_loss, val_acc)):
                this_acc_at_conver, this_epoch_at_conver = get_acc_at_conver(this_val_acc, this_val_loss)
                acc_at_conver[idx, :] = this_acc_at_conver
                epoch_at_conver[idx, :] = this_epoch_at_conver
                # print('sample size %d is done' % idx)
            conver['acc'][df_loc] = np.transpose(acc_at_conver)
            conver['epoch'][df_loc] = np.transpose(epoch_at_conver)
    return conver

In [ ]:
def get_stack_flops(flops, typeStr, acc, colName, networkType_list):
    return np.squeeze(np.stack([np.stack(flops.query(dict_to_query({'typeStr':'\''+typeStr+'\'', 'acc':acc, 'networkType':'\''+networkType +'\''}))[colName].tolist()) for networkType in networkType_list]))

In [ ]:
def get_stack_conver(conver, typeStr, colName, networkType_list):
    return np.squeeze(np.stack([np.stack(conver.query(dict_to_query({'typeStr':'\''+typeStr+'\'', 'networkType':'\''+networkType +'\''}))[colName].tolist()) for networkType in networkType_list]))

#### Below cell plot for Figure 2 first column

In [ ]:
fontsize=20
resetType_list = ['posRand']
for resetType in resetType_list:
    conver_smart = get_conver(metrics, doWeightFreeze_list, resetType_list, 'pretrained', numRun)
    conver_dumb = get_conver(metrics, doWeightFreeze_list, resetType_list, 'vanilla', numRun)
    conver_finetune = get_conver(metrics, doWeightFreeze_list, resetType_list, 'finetune', numRun)
    fig = plt.figure(figsize=(5,4))

    thisAx = plt.gca()
    doExample = False
    x_coord = np.array(sample_size_list)
    xlim=[np.min(x_coord), np.max(x_coord)]
    ylim=0
    color_list = ['r','b','g']

    for didx, doWeightFreeze in enumerate(doWeightFreeze_list):
        if doWeightFreeze:
            typeStr = 'Freeze sufficient-Polarity'
            typeStr_dict = '\'freeze\''
        else:
            typeStr = 'Fluid'
            typeStr_dict = '\'liquid\''
        
        print(typeStr)
        x_mat = conver_smart.query(dict_to_query({'typeStr':typeStr_dict, 'resetType':'\''+resetType+'\''}))['acc'].tolist()[0]
        plot_median_plus_example((1-x_mat)*100, x_coord, ylim, xlim, typeStr, doExample, thisAx, color=color_list[didx])
        # plot_sem((1-x_mat)*100, x_coord, ylim, xlim, typeStr, thisAx, color=color_list[didx])

    x_mat = conver_dumb.query(dict_to_query({'typeStr':'\'freeze\'', 'resetType':'\''+resetType+'\''}))['acc'].tolist()[0]
    plot_median_plus_example((1-x_mat)*100, x_coord, ylim, xlim, 'Freeze RAND-Polarity', doExample, thisAx, color=color_list[2])
    # plot_sem((1-x_mat)*100, x_coord, ylim, xlim, 'Freeze-uninformed', thisAx, color=color_list[2])

    # plt.plot([100,50000], [50,50], '--', color=[.5,.5,.5])
    plt.plot([100,6000], [20,20], '--', color=[.5,.5,.5])

    simpleaxis(thisAx)
    thisAx.set_xscale('log')
    thisAx.set_xticks([100,1000,6000], ['%d' % x for x in [100,1000,6000]], fontsize=fontsize)
    thisAx.set_yticks([0,25,50], ['%d' % x for x in [0,25,50]], fontsize=fontsize)
    # thisAx.set_xticks([100,5000,50000], ['%d' % x for x in [100,5000,50000]], fontsize=fontsize)
    # thisAx.set_yticks([0,50,100], ['%d' % x for x in [0,50,100]], fontsize=fontsize)
    # thisAx.set_ylim([0,40])
    thisAx.set_xlabel('# samples', fontsize=fontsize)
    thisAx.set_ylabel('Validation error %', fontsize=fontsize)
    handles, labels = thisAx.get_legend_handles_labels()
    fig.text(0.5,1,'Statistical Efficiency', ha='center', fontsize=fontsize)

    by_label = {labels[i]:handles[i] for i in [0,1,2]}
    lgd = thisAx.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1,.99), fancybox=True, framealpha=0, fontsize=fontsize)
    plt.savefig(os.path.join(os.getcwd(), 'sample_efficiency.png'), dpi=400, bbox_inches = 'tight')

#### Below cell plot for Supp Figure A.3 first column

In [ ]:
fontsize=20
resetType_list = ['posRand']
for resetType in resetType_list:
    conver_smart = get_conver(metrics, doWeightFreeze_list, resetType_list, 'pretrained', numRun)
    conver_dumb = get_conver(metrics, doWeightFreeze_list, resetType_list, 'vanilla', numRun)
    conver_finetune = get_conver(metrics, doWeightFreeze_list, resetType_list, 'finetune', numRun)
    fig = plt.figure(figsize=(5,4))

    thisAx = plt.gca()
    doExample = False
    x_coord = np.array(sample_size_list)
    xlim=[np.min(x_coord), np.max(x_coord)]
    ylim=0
    color_list = ['r','b', [0,1,1], [1,.5,0]]

    for didx, doWeightFreeze in enumerate(doWeightFreeze_list):
        if doWeightFreeze:
            typeStr = 'Freeze'
            typeStr_dict = '\'freeze\''
        else:
            typeStr = 'Fluid'
            typeStr_dict = '\'liquid\''
        
        print(typeStr)
        x_mat = conver_smart.query(dict_to_query({'typeStr':typeStr_dict, 'resetType':'\''+resetType+'\''}))['acc'].tolist()[0]
        if doWeightFreeze:
            typeStrTmp = typeStr+' IN-Polarity'
        else:
            typeStrTmp = typeStr
        plot_median_plus_example((1-x_mat)*100, x_coord, ylim, xlim, typeStrTmp, doExample, thisAx, color=color_list[didx])
        # plot_sem((1-x_mat)*100, x_coord, ylim, xlim, typeStr, thisAx, color=color_list[didx])

        x_mat = conver_finetune.query(dict_to_query({'typeStr':typeStr_dict, 'resetType':'\''+resetType+'\''}))['acc'].tolist()[0]
        plot_median_plus_example((1-x_mat)*100, x_coord, ylim, xlim, typeStr+' IN-Weight', doExample, thisAx, color=color_list[didx+2], lineType = '--')
        # plot_sem((1-x_mat)*100, x_coord, ylim, xlim, typeStr, thisAx, color=color_list[didx], lineType = '--')

    # plt.plot([100,50000], [50,50], '--', color=[.5,.5,.5])
    # plt.plot([100,6000], [20,20], '--', color=[.5,.5,.5])

    simpleaxis(thisAx)
    thisAx.set_xscale('log')
    thisAx.set_xticks([100,1000,6000], ['%d' % x for x in [100,1000,6000]], fontsize=fontsize)
    thisAx.set_yticks([0,25,50], ['%d' % x for x in [0,25,50]], fontsize=fontsize)
    # thisAx.set_xticks([100,5000,50000], ['%d' % x for x in [100,5000,50000]], fontsize=fontsize)
    # thisAx.set_yticks([0,50,100], ['%d' % x for x in [0,50,100]], fontsize=fontsize)
    # thisAx.set_ylim([0,40])
    thisAx.set_xlabel('# samples', fontsize=fontsize)
    thisAx.set_ylabel('Validation error %', fontsize=fontsize)
    handles, labels = thisAx.get_legend_handles_labels()
    fig.text(0.5,1,'Statistical Efficiency', ha='center', fontsize=fontsize)

    by_label = {labels[i]:handles[i] for i in [0,3,1,2]}
    lgd = thisAx.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1,.99), fancybox=True, framealpha=0, fontsize=fontsize)
    plt.savefig(os.path.join(os.getcwd(), 'sample_efficiency_finetune.png'), dpi=400, bbox_inches = 'tight')

#### Below cell plot for Figure 3 first column

In [ ]:
fontsize=20
resetType_list = ['posRand']
for resetType in resetType_list:
    conver_smart = get_conver(metrics, doWeightFreeze_list, resetType_list, 'pretrained', numRun)
    conver_dumb = get_conver(metrics, doWeightFreeze_list, resetType_list, 'vanilla', numRun)
    conver_finetune = get_conver(metrics, doWeightFreeze_list, resetType_list, 'finetune', numRun)
    fig = plt.figure(figsize=(5,4))

    thisAx = plt.gca()
    doExample = False
    x_coord = np.array(sample_size_list)
    xlim=[np.min(x_coord), np.max(x_coord)]
    ylim=0
    color_list = ['b', [0,1,1], [1,.5,0]]

    base_line = conver_smart.query(dict_to_query({'typeStr':'\'freeze\'', 'resetType':'\''+resetType+'\''}))['acc'].tolist()[0]

    y_mat = conver_smart.query(dict_to_query({'typeStr':'\'liquid\'', 'resetType':'\''+resetType+'\''}))['acc'].tolist()[0]
    plot_diff_plus_mannwhitneyu((1-base_line)*100, (1-y_mat)*100, x_coord, ylim, xlim, 'Liquid RAND-Polarity', thisAx, color=color_list[0], linewidth=4)

    y_mat = conver_finetune.query(dict_to_query({'typeStr':'\'freeze\'', 'resetType':'\''+resetType+'\''}))['acc'].tolist()[0]
    plot_diff_plus_mannwhitneyu((1-base_line)*100, (1-y_mat)*100, x_coord, ylim, xlim, 'Freeze IN-Weight', thisAx, color=color_list[1], linewidth=4)

    y_mat = conver_finetune.query(dict_to_query({'typeStr':'\'liquid\'', 'resetType':'\''+resetType+'\''}))['acc'].tolist()[0]
    plot_diff_plus_mannwhitneyu((1-base_line)*100, (1-y_mat)*100, x_coord, ylim, xlim, 'Liquid IN-Weight', thisAx, color=color_list[2], linewidth=4)

    plt.plot([90,50000], [0,0], '--', color=[.5,.5,.5])
    # plt.plot([90,6000], [0,0], '--', color=[.5,.5,.5])

    simpleaxis(thisAx)
    thisAx.set_xscale('log')
    # thisAx.set_xticks([100,1000,6000], ['%d' % x for x in [100,1000,6000]], fontsize=fontsize)
    # thisAx.set_yticks([0,10], ['%d' % x for x in [0,10]], fontsize=fontsize)
    # thisAx.set_xlim([90,6000])
    thisAx.set_xticks([100,5000,50000], ['%d' % x for x in [100,5000,50000]], fontsize=fontsize)
    thisAx.set_yticks([0,25], ['%d' % x for x in [0,25]], fontsize=fontsize)
    thisAx.set_xlim([90,50000])
    # thisAx.set_ylim([0,40])
    thisAx.set_xlabel('# samples', fontsize=fontsize)
    thisAx.set_ylabel('$\Delta$ Validation error %', fontsize=fontsize)
    handles, labels = thisAx.get_legend_handles_labels()
    fig.text(0.5,1,'Statistical Efficiency', ha='center', fontsize=fontsize)

    by_label = {labels[i]:handles[i] for i in [2,1,0]}
    lgd = thisAx.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1,.99), fancybox=True, framealpha=0, fontsize=fontsize)
    plt.savefig(os.path.join(os.getcwd(), 'sample_efficiency_test.png'), dpi=400, bbox_inches = 'tight')

### 4.**TIME: Computational Efficiency** - Figure 2 & 3 third column

#### Load func

In [ ]:
def get_learnt_epoch(x, cutoff):
    epoch_num = np.squeeze(np.where(x >= cutoff))
    if not len(epoch_num.shape) == 0: # only has single element
        if epoch_num.shape[0]==0:
            epoch_num = np.NaN
        else:
            epoch_num = epoch_num[0].astype(np.float64)
    else:
        epoch_num = epoch_num.astype(np.float64)
    return epoch_num

In [ ]:
def get_flops(metrics, doWeightFreeze_list, acc_cutoff_list, resetType_list, networkType, numRun):
    flops = pd.DataFrame(index=range(len(doWeightFreeze_list)*len(acc_cutoff_list)*len(resetType_list)), columns=["typeStr", "resetType", "networkType", "acc", "epoch", "flops"])
    counter = 0

    for resetType in resetType_list:
        for acc_cutoff in acc_cutoff_list:
            for doWeightFreeze in doWeightFreeze_list:
                if doWeightFreeze:
                    typeStr = 'freeze'
                    typeStr_dict = '\'freeze\''
                else:
                    typeStr = 'liquid'
                    typeStr_dict = '\'liquid\''
                flops['typeStr'][counter] = typeStr
                flops['acc'][counter] = acc_cutoff
                flops['resetType'][counter] = resetType
                flops['networkType'][counter] = networkType

                # get different sample sizes into list
                page_dict = {'networkType':'\'' + networkType + '\'', 'typeStr':typeStr_dict, 'resetType':'\'' + resetType + '\''} #, 'train_sample':sample_size_list[8]
                val_acc = metrics.query(dict_to_query(page_dict))['validation_acc'].tolist()
                # print(val_acc[0].shape)

                epoch_at_learnt = np.empty(shape=(len(sample_size_list), numRun), dtype=np.float64)
                for idx, this_val_acc in enumerate(val_acc):
                    this_epoch_at_learnt = np.apply_along_axis(get_learnt_epoch, 1, this_val_acc, acc_cutoff)
                    epoch_at_learnt[idx, :] = np.squeeze(this_epoch_at_learnt)
                    del this_epoch_at_learnt
                flops['epoch'][counter] = np.transpose(epoch_at_learnt)
                flops['flops'][counter] = flops['epoch'][counter]*62388354 # hSize_infocus*3
                counter+=1
        # print('@acc %1.2f is done' % acc_cutoff)
    return flops

#### Below cell plot for Figure 2 third column & Supp Figure A.2

In [ ]:
# acc_cutoff_list=[.5,.6,.75, 0.8, .82, .85]
acc_cutoff_list=[.45,.5,.55,.6,.75, 0.8, .82, .85]
# acc_cutoff_list=[.3,.4,.5,.6]
colName = 'epoch'
doTitle = False
resetType_list = ['posRand']
for resetType in resetType_list:
    flops_smart = get_flops(metrics, doWeightFreeze_list, acc_cutoff_list, resetType_list, 'pretrained', numRun)
    flops_dumb = get_flops(metrics, doWeightFreeze_list, acc_cutoff_list, resetType_list, 'vanilla', numRun)
    flops_finetune = get_flops(metrics, doWeightFreeze_list, acc_cutoff_list, resetType_list, 'finetune', numRun)

    fontsize=20
    doExample = False
    x_coord = np.array(sample_size_list)
    xlim=[100, np.max(x_coord)]
    ylim=0
    lineTypeList = ['-', '--']
    lineType = '-'
    color_list = ['r','b','g']

    for acc_cutoff in acc_cutoff_list:
        fig = plt.figure(figsize=(5,4))
        thisAx = plt.gca()
    
        for doWeightFreeze, color in zip(doWeightFreeze_list, color_list[:2]):
            if doWeightFreeze:
                typeStr = 'Freeze_informed'
                typeStr_dict = '\'freeze\''
            else:
                typeStr = 'Fluid'
                typeStr_dict = '\'liquid\''

            x_mat = flops_smart.query(dict_to_query({'typeStr':typeStr_dict, 'acc':acc_cutoff, 'resetType': '\''+ resetType + '\''}))[colName].tolist()[0]
            plot_median_plus_example(x_mat, x_coord, ylim, xlim, typeStr, doExample, thisAx, color=color, lineType=lineType)

        x_mat = flops_dumb.query(dict_to_query({'typeStr':'\'freeze\'', 'acc':acc_cutoff, 'resetType': '\''+ resetType + '\''}))[colName].tolist()[0]
        plot_median_plus_example(x_mat, x_coord, ylim, xlim, 'Freeze_uninformed', doExample, thisAx, color=color_list[2], lineType=lineType)

        simpleaxis(thisAx)
        thisAx.set_xscale('log')
        thisAx.set_xlim(xlim)
        thisAx.set_xticks([100,1000,6000], ['%d' % x for x in [100,1000,6000]], fontsize=fontsize)
        # thisAx.set_xticks([100,5000,50000], ['%d' % x for x in [100,5000,50000]], fontsize=fontsize)

        y_min, y_max = thisAx.get_ylim()
        thisAx.set_yticks([0,round(y_max, -1)/2,round(y_max, -1)], ['%d' % x for x in [0,round(y_max, -1)/2,round(y_max, -1)]], fontsize=fontsize)
        thisAx.set_xlabel('# samples', fontsize=fontsize)
        thisAx.set_ylabel('#'+colName+'s to\n reach %d%% accuracy' % int(acc_cutoff*100), fontsize=fontsize)

        if doTitle:
            y_min, y_max = thisAx.get_ylim()
            thisAx.text(100, y_max*1.05, resetType,fontweight="bold",fontsize=fontsize, ha='center')

        handles, labels = thisAx.get_legend_handles_labels()
        fig.text(0.5,1,'Computational Efficiency', ha='center', fontsize=fontsize)

        # by_label = {labels[i]:handles[i] for i in [0,2,1]}
        # lgd = thisAx.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1,.99), fancybox=True, framealpha=0, fontsize=25)
        plt.savefig(os.path.join(os.getcwd(), 'computational_efficiency_%1.2f.png' % acc_cutoff), dpi=400, bbox_inches = 'tight')

#### Below cell plot for Supp Figure 3 third column

In [ ]:
# acc_cutoff_list=[.5,.6,.75, 0.8, .82, .85]
acc_cutoff_list=[.45,.5,.55,.6,.75, 0.8, .82, .85]
# acc_cutoff_list=[.3,.4,.5,.6]
colName = 'epoch'
doTitle = False
resetType_list = ['posRand']
for resetType in resetType_list:
    flops_smart = get_flops(metrics, doWeightFreeze_list, acc_cutoff_list, resetType_list, 'pretrained', numRun)
    flops_dumb = get_flops(metrics, doWeightFreeze_list, acc_cutoff_list, resetType_list, 'vanilla', numRun)
    flops_finetune = get_flops(metrics, doWeightFreeze_list, acc_cutoff_list, resetType_list, 'finetune', numRun)

    fontsize=20
    doExample = False
    x_coord = np.array(sample_size_list)
    xlim=[100, np.max(x_coord)]
    ylim=0
    lineTypeList = ['-', '--']
    lineType = '-'
    color_list = ['r','b', [0,1,1], [1,.5,0]]
    
    for acc_cutoff in acc_cutoff_list:
        fig = plt.figure(figsize=(5,4))
        thisAx = plt.gca()
    
        for didx, doWeightFreeze in enumerate(doWeightFreeze_list):
            if doWeightFreeze:
                typeStr = 'Freeze_informed'
                typeStr_dict = '\'freeze\''
            else:
                typeStr = 'Fluid'
                typeStr_dict = '\'liquid\''

            x_mat = flops_smart.query(dict_to_query({'typeStr':typeStr_dict, 'acc':acc_cutoff, 'resetType': '\''+ resetType + '\''}))[colName].tolist()[0]
            plot_median_plus_example(x_mat, x_coord, ylim, xlim, typeStr, doExample, thisAx, color=color_list[didx], lineType=lineType)
            x_mat = flops_finetune.query(dict_to_query({'typeStr':typeStr_dict, 'acc':acc_cutoff, 'resetType': '\''+ resetType + '\''}))[colName].tolist()[0]
            plot_median_plus_example(x_mat, x_coord, ylim, xlim, typeStr+'-finetune', doExample, thisAx, color=color_list[didx+2], lineType='--')

        simpleaxis(thisAx)
        thisAx.set_xscale('log')
        thisAx.set_xlim(xlim)
        thisAx.set_xticks([100,1000,6000], ['%d' % x for x in [100,1000,6000]], fontsize=fontsize)
        # thisAx.set_xticks([100,5000,50000], ['%d' % x for x in [100,5000,50000]], fontsize=fontsize)

        y_min, y_max = thisAx.get_ylim()
        thisAx.set_yticks([0,round(y_max, -1)/2,round(y_max, -1)], ['%d' % x for x in [0,round(y_max, -1)/2,round(y_max, -1)]], fontsize=fontsize)
        thisAx.set_xlabel('# samples', fontsize=fontsize)
        thisAx.set_ylabel('#'+colName+'s to\n reach %d%% accuracy' % int(acc_cutoff*100), fontsize=fontsize)

        if doTitle:
            y_min, y_max = thisAx.get_ylim()
            thisAx.text(100, y_max*1.05, resetType,fontweight="bold",fontsize=fontsize, ha='center')

        handles, labels = thisAx.get_legend_handles_labels()
        fig.text(0.5,1,'Computational Efficiency', ha='center', fontsize=fontsize)

        # by_label = {labels[i]:handles[i] for i in [0,2,3,1]}
        # lgd = thisAx.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1,.99), fancybox=True, framealpha=0, fontsize=25)
        plt.savefig(os.path.join(os.getcwd(), 'computational_efficiency_%1.2f_finetune.png' % acc_cutoff), dpi=400, bbox_inches = 'tight')

#### Below cell plot for Figure 3 third column & Supp Figure A.4

In [ ]:
# acc_cutoff_list=[.5,.6,.75, 0.8, .82, .85]
acc_cutoff_list=[.45,.5,.55,.6,.75, 0.8, .82, .85]
# acc_cutoff_list=[.3,.4,.5,.6]
colName = 'epoch'
doTitle = False
resetType_list = ['posRand']
for resetType in resetType_list:
    flops_smart = get_flops(metrics, doWeightFreeze_list, acc_cutoff_list, resetType_list, 'pretrained', numRun)
    flops_dumb = get_flops(metrics, doWeightFreeze_list, acc_cutoff_list, resetType_list, 'vanilla', numRun)
    flops_finetune = get_flops(metrics, doWeightFreeze_list, acc_cutoff_list, resetType_list, 'finetune', numRun)

    fontsize=20
    doExample = False
    x_coord = np.array(sample_size_list)
    xlim=[100, np.max(x_coord)]
    ylim=0
    lineTypeList = ['-', '--']
    lineType = '-'
    color_list = ['b', [0,1,1], [1,.5,0]]
    
    for acc_cutoff in acc_cutoff_list:
        fig = plt.figure(figsize=(5,4))
        thisAx = plt.gca()

        base_line = flops_smart.query(dict_to_query({'typeStr':'\'freeze\'', 'acc':acc_cutoff, 'resetType': '\''+ resetType + '\''}))[colName].tolist()[0]

        y_mat = flops_smart.query(dict_to_query({'typeStr':'\'liquid\'', 'acc':acc_cutoff, 'resetType': '\''+ resetType + '\''}))[colName].tolist()[0]
        plot_diff_plus_mannwhitneyu(base_line, y_mat, x_coord, ylim, xlim, 'Liquid RAND-Polarity', thisAx, color=color_list[0], linewidth=4)

        y_mat = flops_finetune.query(dict_to_query({'typeStr':'\'freeze\'', 'acc':acc_cutoff, 'resetType': '\''+ resetType + '\''}))[colName].tolist()[0]
        plot_diff_plus_mannwhitneyu(base_line, y_mat, x_coord, ylim, xlim, 'Freeze IN-Weight', thisAx, color=color_list[1], linewidth=4)

        y_mat = flops_finetune.query(dict_to_query({'typeStr':'\'liquid\'', 'acc':acc_cutoff, 'resetType': '\''+ resetType + '\''}))[colName].tolist()[0]
        plot_diff_plus_mannwhitneyu(base_line, y_mat, x_coord, ylim, xlim, 'Liquid IN-Weight', thisAx, color=color_list[2], linewidth=4)

        # plt.plot([100,50000], [0,0], '--', color=[.5,.5,.5])
        plt.plot([100,6000], [0,0], '--', color=[.5,.5,.5])

        simpleaxis(thisAx)
        thisAx.set_xscale('log')
        thisAx.set_xlim(xlim)
        thisAx.set_xticks([100,1000,6000], ['%d' % x for x in [100,1000,6000]], fontsize=fontsize)
        # thisAx.set_xticks([100,5000,50000], ['%d' % x for x in [100,5000,50000]], fontsize=fontsize)

        y_min, y_max = thisAx.get_ylim()
        thisAx.set_yticks([0,round(y_max, -1)/2,round(y_max, -1)], ['%d' % x for x in [0,round(y_max, -1)/2,round(y_max, -1)]], fontsize=fontsize)
        thisAx.set_xlabel('# samples', fontsize=fontsize)
        thisAx.set_ylabel('$\Delta$ #'+colName+'s to\n reach %d%% accuracy' % int(acc_cutoff*100), fontsize=fontsize)

        if doTitle:
            y_min, y_max = thisAx.get_ylim()
            thisAx.text(100, y_max*1.05, resetType,fontweight="bold",fontsize=fontsize, ha='center')

        handles, labels = thisAx.get_legend_handles_labels()
        fig.text(0.5,1,'Computational Efficiency', ha='center', fontsize=fontsize)

        # by_label = {labels[i]:handles[i] for i in [0,2,3,1]}
        # lgd = thisAx.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1,.99), fancybox=True, framealpha=0, fontsize=25)
        plt.savefig(os.path.join(os.getcwd(), 'computational_efficiency_diff_%1.2f_finetune.png' % acc_cutoff), dpi=400, bbox_inches = 'tight')

### 5.**PROBABILITY** - Figure 2&3 second column

#### Load func

In [ ]:
def get_prob_sim(metrics, numRun, typeStr, sample_size_list, acc_cutoff):
    return np.stack([[np.sum(np.any(this_metric>acc_cutoff, axis=1))/numRun for this_metric in metrics.query(dict_to_query({'resetType':'\'posRand\'', 'typeStr':'\''+typeStr+'\'', 'train_sample':train_sample}))['validation_acc'].tolist()] for train_sample in sample_size_list])

#### Below cell plot for Figure 2 second column & Supp Figure A.2

In [ ]:
resetType_list = ['posRand']
lineStyle_list = ['-']
acc_cutoff_list=[.45,.5,.55,.6,.75, 0.8, .82, .85]
# acc_cutoff_list=[.5,.6,.75, 0.8, .82,.85]
# acc_cutoff_list=[.3,.4,.5,.6]

for acc_cutoff in acc_cutoff_list:
    for resetType in resetType_list:
        fig = plt.figure(figsize=(5,4))
        fontsize=20
        x_l = sample_size_list

        y_l = {}
        y_l['Freeze_informed'] = [np.sum(np.any(this_metric>acc_cutoff, axis=1))/numRun for this_metric in metrics.query(dict_to_query({'networkType':'\'pretrained\'', 'resetType':'\''+resetType+'\'', 'typeStr':'\'freeze\''}))['validation_acc'].tolist()]
        y_l['Fluid'] = [np.sum(np.any(this_metric>acc_cutoff, axis=1))/numRun for this_metric in metrics.query(dict_to_query({'networkType':'\'pretrained\'', 'resetType':'\''+resetType+'\'', 'typeStr':'\'liquid\''}))['validation_acc'].tolist()]

        plt.plot(x_l, np.array(y_l['Freeze_informed'])*100, label='Freeze_informed', color='r', linewidth=4)
        plt.plot(x_l, np.array(y_l['Fluid'])*100, label='Fluid', color='b', linewidth=4)

        for lineStyle in lineStyle_list:
            y_l['Freeze_uninformed'] = [np.sum(np.any(this_metric>acc_cutoff, axis=1))/numRun for this_metric in metrics.query(dict_to_query({'networkType':'\'vanilla\'', 'resetType':'\''+resetType+'\'', 'typeStr':'\'freeze\''}))['validation_acc'].tolist()]
            plt.plot(x_l, np.array(y_l['Freeze_uninformed'])*100, lineStyle, label='Freeze_uninformed', color='g', linewidth=4)

        plt.xlabel('# samples', fontsize=fontsize)
        plt.ylabel('%% trials reached\n %d%% validation accuracy' % int(acc_cutoff*100), fontsize=fontsize)

        plt.gca().set_xscale('log')
        plt.gca().set_xticks([100,1000, 6000], ['%d' % x for x in [100,1000, 6000]], fontsize=fontsize)#x_coord
        # plt.gca().set_xticks([100,5000, 50000], ['%d' % x for x in [100,5000, 50000]], fontsize=fontsize)#x_coord
        plt.gca().set_xlim([100, max(sample_size_list)])
        plt.yticks([0,100], fontsize=fontsize)
        # plt.grid(True)
        plt.gca().spines['top'].set_visible(False)
        plt.gca().spines['right'].set_visible(False)
        # plt.gca().legend(loc='upper right', bbox_to_anchor=[1,.95], fancybox=True, framealpha=0)
        handles, labels = plt.gca().get_legend_handles_labels()
        rect = plt.gca().patch
        rect.set_alpha(0)
        # fig.text(.5,1,'$P$(learning XOR)', ha='center', fontsize=fontsize)

        # by_label = {labels[i]:handles[i] for i in [0,1,2]}
        # lgd = plt.gca().legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1,.99), fancybox=True, framealpha=0, fontsize=fontsize)
        plt.savefig(os.path.join(os.getcwd(), 'prob_learning_%1.2f.png' % acc_cutoff), dpi=400, bbox_inches = 'tight')

#### Below cell plot for Figure 3 second column & Supp Figure A.3-4

In [ ]:
resetType_list = ['posRand']
lineStyle_list = ['-']
# acc_cutoff_list=[.5,.6,.75, 0.8, .82,.85]
# acc_cutoff_list=[.3,.4,.5,.6]

for acc_cutoff in acc_cutoff_list:
    for resetType in resetType_list:
        fig = plt.figure(figsize=(5,4))
        fontsize=20
        x_l = sample_size_list

        y_l = {}
        y_l['Freeze_informed'] = [np.sum(np.any(this_metric>acc_cutoff, axis=1))/numRun for this_metric in metrics.query(dict_to_query({'networkType':'\'pretrained\'', 'resetType':'\''+resetType+'\'', 'typeStr':'\'freeze\''}))['validation_acc'].tolist()]
        y_l['Fluid'] = [np.sum(np.any(this_metric>acc_cutoff, axis=1))/numRun for this_metric in metrics.query(dict_to_query({'networkType':'\'pretrained\'', 'resetType':'\''+resetType+'\'', 'typeStr':'\'liquid\''}))['validation_acc'].tolist()]

        plt.plot(x_l, np.array(y_l['Freeze_informed'])*100, label='Freeze_informed', color='r', linewidth=4)
        plt.plot(x_l, np.array(y_l['Fluid'])*100, label='Fluid', color='b', linewidth=4)

        y_l['Freeze_finetune'] = [np.sum(np.any(this_metric>acc_cutoff, axis=1))/numRun for this_metric in metrics.query(dict_to_query({'networkType':'\'finetune\'', 'resetType':'\''+resetType+'\'', 'typeStr':'\'freeze\''}))['validation_acc'].tolist()]
        y_l['Fluid_finetune'] = [np.sum(np.any(this_metric>acc_cutoff, axis=1))/numRun for this_metric in metrics.query(dict_to_query({'networkType':'\'finetune\'', 'resetType':'\''+resetType+'\'', 'typeStr':'\'liquid\''}))['validation_acc'].tolist()]
        plt.plot(x_l, np.array(y_l['Freeze_finetune'])*100, label='Freeze_finetune', color=[0,1,1], linestyle = '--', linewidth=4)
        plt.plot(x_l, np.array(y_l['Fluid_finetune'])*100, label='Fluid_finetune', color=[1,.5,0], linestyle = '--', linewidth=4)

        plt.xlabel('# samples', fontsize=fontsize)
        plt.ylabel('%% trials reached\n %d%% validation accuracy' % int(acc_cutoff*100), fontsize=fontsize)

        plt.gca().set_xscale('log')
        plt.gca().set_xticks([100,1000, 6000], ['%d' % x for x in [100,1000, 6000]], fontsize=fontsize)#x_coord
        # plt.gca().set_xticks([100,5000, 50000], ['%d' % x for x in [100,5000, 50000]], fontsize=fontsize)#x_coord
        plt.yticks([0,100], fontsize=fontsize)
        plt.gca().set_xlim([100, max(sample_size_list)])
        # plt.grid(True)
        plt.gca().spines['top'].set_visible(False)
        plt.gca().spines['right'].set_visible(False)
        # plt.gca().legend(loc='upper right', bbox_to_anchor=[1,.95], fancybox=True, framealpha=0)
        handles, labels = plt.gca().get_legend_handles_labels()
        rect = plt.gca().patch
        rect.set_alpha(0)
        # fig.text(.5,1,'$P$(learning XOR)', ha='center', fontsize=fontsize)

        # by_label = {labels[i]:handles[i] for i in [0,1,2,3]}
        # lgd = plt.gca().legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1,.99), fancybox=True, framealpha=0, fontsize=fontsize)
        plt.savefig(os.path.join(os.getcwd(), 'prob_learning_%1.2f_finetune.png' % acc_cutoff), dpi=400, bbox_inches = 'tight')